# [1321. Restaurant Growth](https://leetcode.com/problems/restaurant-growth/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Customer

<pre>+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| customer_id   | int     |
| name          | varchar |
| visited_on    | date    |
| amount        | int     |
+---------------+---------+</pre>
In SQL,(customer_id, visited_on) is the primary key for this table.
This table contains data about customer transactions in a restaurant.
visited_on is the date on which the customer with ID (customer_id) has visited the restaurant.
amount is the total paid by a customer.
 

You are the restaurant owner and you want to analyze a possible expansion (there will be at least one customer every day).

Compute the moving average of how much the customer paid in a seven days window (i.e., current day + 6 days before). average_amount should be rounded to two decimal places.

Return the result table ordered by visited_on in ascending order.

The result format is in the following example.

 

Example 1:

Input: 
Customer table:
<pre>+-------------+--------------+--------------+-------------+
| customer_id | name         | visited_on   | amount      |
+-------------+--------------+--------------+-------------+
| 1           | Jhon         | 2019-01-01   | 100         |
| 2           | Daniel       | 2019-01-02   | 110         |
| 3           | Jade         | 2019-01-03   | 120         |
| 4           | Khaled       | 2019-01-04   | 130         |
| 5           | Winston      | 2019-01-05   | 110         | 
| 6           | Elvis        | 2019-01-06   | 140         | 
| 7           | Anna         | 2019-01-07   | 150         |
| 8           | Maria        | 2019-01-08   | 80          |
| 9           | Jaze         | 2019-01-09   | 110         | 
| 1           | Jhon         | 2019-01-10   | 130         | 
| 3           | Jade         | 2019-01-10   | 150         | 
+-------------+--------------+--------------+-------------+</pre>
Output: 
<pre>+--------------+--------------+----------------+
| visited_on   | amount       | average_amount |
+--------------+--------------+----------------+
| 2019-01-07   | 860          | 122.86         |
| 2019-01-08   | 840          | 120            |
| 2019-01-09   | 840          | 120            |
| 2019-01-10   | 1000         | 142.86         |
+--------------+--------------+----------------+</pre>
Explanation: 
1st moving average from 2019-01-01 to 2019-01-07 has an average_amount of (100 + 110 + 120 + 130 + 110 + 140 + 150)/7 = 122.86
2nd moving average from 2019-01-02 to 2019-01-08 has an average_amount of (110 + 120 + 130 + 110 + 140 + 150 + 80)/7 = 120
3rd moving average from 2019-01-03 to 2019-01-09 has an average_amount of (120 + 130 + 110 + 140 + 150 + 80 + 110)/7 = 120
4th moving average from 2019-01-04 to 2019-01-10 has an average_amount of (130 + 110 + 140 + 150 + 80 + 110 + 130 + 150)/7 = 142.86

In [1]:
import datetime

# pandas schema

import pandas as pd

data = [[1, 'Jhon', '2019-01-01', 100], [2, 'Daniel', '2019-01-02', 110], [3, 'Jade', '2019-01-03', 120],
        [4, 'Khaled', '2019-01-04', 130], [5, 'Winston', '2019-01-05', 110], [6, 'Elvis', '2019-01-06', 140],
        [7, 'Anna', '2019-01-07', 150], [8, 'Maria', '2019-01-08', 80], [9, 'Jaze', '2019-01-09', 110],
        [1, 'Jhon', '2019-01-10', 130], [3, 'Jade', '2019-01-10', 150]]
       # [1, 'Jhon', '2019-01-10', 130], [3, 'Jade', '2019-01-10', 150],[3, 'Jade', '2019-01-12', 150]]
customer = pd.DataFrame(data, columns=['customer_id', 'name', 'visited_on', 'amount']).astype(
    {'customer_id': 'Int64', 'name': 'object', 'visited_on': 'datetime64[ns]', 'amount': 'Int64'})

In [2]:
#to spark dataframe
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

customer_df = spark.createDataFrame(customer)
customer_df.show()

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
23/11/06 15:43:35 WARN Utils: Your hostname, svchost resolves to a loopback address: 127.0.1.1; using 172.18.28.34 instead (on interface eth0)
23/11/06 15:43:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/06 15:43:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-----------+-------+-------------------+------+
|customer_id|   name|         visited_on|amount|
+-----------+-------+-------------------+------+
|          1|   Jhon|2019-01-01 00:00:00|   100|
|          2| Daniel|2019-01-02 00:00:00|   110|
|          3|   Jade|2019-01-03 00:00:00|   120|
|          4| Khaled|2019-01-04 00:00:00|   130|
|          5|Winston|2019-01-05 00:00:00|   110|
|          6|  Elvis|2019-01-06 00:00:00|   140|
|          7|   Anna|2019-01-07 00:00:00|   150|
|          8|  Maria|2019-01-08 00:00:00|    80|
|          9|   Jaze|2019-01-09 00:00:00|   110|
|          1|   Jhon|2019-01-10 00:00:00|   130|
|          3|   Jade|2019-01-10 00:00:00|   150|
+-----------+-------+-------------------+------+


In [3]:
from pyspark.sql import Window
# Solving in Spark DataFrame
import pyspark.sql.functions as F

min_max_dates = customer_df.agg(F.min('visited_on'),F.max('visited_on'))
min_max_dates.show()

+-------------------+-------------------+
|    min(visited_on)|    max(visited_on)|
+-------------------+-------------------+
|2019-01-01 00:00:00|2019-01-10 00:00:00|
+-------------------+-------------------+


In [4]:
from datetime import timedelta

min_date = min_max_dates.collect()[0][0]
max_date = min_max_dates.collect()[0][1]

days_diff = max_date-min_date
from_min_to_max_date_list = [(min_date + timedelta(days=i),0) for i in range(0,days_diff.days+1)]

dummy_df_header = ['visited_on','amount']
dummy_df = spark.createDataFrame(from_min_to_max_date_list,dummy_df_header)
dummy_df.show()

+-------------------+------+
|         visited_on|amount|
+-------------------+------+
|2019-01-01 00:00:00|     0|
|2019-01-02 00:00:00|     0|
|2019-01-03 00:00:00|     0|
|2019-01-04 00:00:00|     0|
|2019-01-05 00:00:00|     0|
|2019-01-06 00:00:00|     0|
|2019-01-07 00:00:00|     0|
|2019-01-08 00:00:00|     0|
|2019-01-09 00:00:00|     0|
|2019-01-10 00:00:00|     0|
+-------------------+------+


In [5]:
df = customer_df \
    .select('visited_on','amount')\
    .unionAll(dummy_df)\
    .groupBy('visited_on') \
    .agg(F.sum('amount').alias('amount')) \
    .withColumn('amount', F.sum('amount').over(Window.orderBy('visited_on').rowsBetween(-6, 0)))\
    .withColumn('average_amount', F.round(F.col('amount')/7,2))\
    .withColumn('rn', F.row_number().over(Window.orderBy('visited_on')))\
    .filter(F.col('rn')>=7)\
    .select('visited_on','amount','average_amount')\
    .orderBy('visited_on')
df.show()

23/11/06 15:43:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/06 15:43:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/06 15:43:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/06 15:43:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/06 15:43:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/06 15:43:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/06 1

+-------------------+------+--------------+
|         visited_on|amount|average_amount|
+-------------------+------+--------------+
|2019-01-07 00:00:00|   860|        122.86|
|2019-01-08 00:00:00|   840|         120.0|
|2019-01-09 00:00:00|   840|         120.0|
|2019-01-10 00:00:00|  1000|        142.86|
+-------------------+------+--------------+


In [6]:
# Solving in spark SQL

customer_df.createOrReplaceTempView('customer')
# this solution will not work if there is missing visited_on between start and end date.
spark.sql('''
select visited_on, amount, average_amount from
(select 
    visited_on, 
    sum(amount) over (order by visited_on rows between 6 preceding and current row) as amount,
    round(avg(amount) over (order by visited_on rows between 6 preceding and current row),2) as average_amount,
    row_number() over (order by visited_on) as rn
from (
    select visited_on, sum(amount) as amount
    from customer
    group by visited_on
)) where rn>=7;
''').show()

23/11/06 15:44:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/06 15:44:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/06 15:44:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/06 15:44:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/06 15:44:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/06 15:44:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/06 1

+-------------------+------+--------------+
|         visited_on|amount|average_amount|
+-------------------+------+--------------+
|2019-01-07 00:00:00|   860|        122.86|
|2019-01-08 00:00:00|   840|         120.0|
|2019-01-09 00:00:00|   840|         120.0|
|2019-01-10 00:00:00|  1000|        142.86|
+-------------------+------+--------------+


In [7]:
## this mysql solution will work for the data with missing dates as well

'''
 WITH RECURSIVE date_range AS (
  SELECT (select min(visited_on) from customer) AS visited_on, 0 as amount
  UNION ALL
  SELECT DATE_ADD(visited_on, INTERVAL 1 DAY), 0 as amount
  FROM date_range
  WHERE DATE_ADD(visited_on, INTERVAL 1 DAY) <= (select max(visited_on) from customer)
), temp_table as (
    select * from date_range
    union all
    select visited_on, amount as amount from customer
), tbl as (
    select 
        visited_on, 
        sum(amount) over (order by visited_on rows between 6 preceding and current row) as amount,
        round(avg(amount) over (order by visited_on rows between 6 preceding and current row),2) as average_amount,
        row_number() over (order by visited_on) as rn
    from (
        select visited_on, sum(amount) as amount
        from temp_table
        group by visited_on
    ) t
)
select visited_on, amount, average_amount
from tbl
where rn >=7;
'''

'\n WITH RECURSIVE date_range AS (\n  SELECT (select min(visited_on) from customer) AS visited_on, 0 as amount\n  UNION ALL\n  SELECT DATE_ADD(visited_on, INTERVAL 1 DAY), 0 as amount\n  FROM date_range\n  WHERE DATE_ADD(visited_on, INTERVAL 1 DAY) <= (select max(visited_on) from customer)\n), temp_table as (\n    select * from date_range\n    union all\n    select visited_on, amount as amount from customer\n), tbl as (\n    select \n        visited_on, \n        sum(amount) over (order by visited_on rows between 6 preceding and current row) as amount,\n        round(avg(amount) over (order by visited_on rows between 6 preceding and current row),2) as average_amount,\n        row_number() over (order by visited_on) as rn\n    from (\n        select visited_on, sum(amount) as amount\n        from temp_table\n        group by visited_on\n    ) t\n)\nselect visited_on, amount, average_amount\nfrom tbl\nwhere rn >=7;\n'